In [1]:
import pandas as pd
import numpy as np
%load_ext Cython

## python version

In [2]:
%quickref

In [3]:
%%time
def int_isnan(x):
    return np.isnan(float(x))

def do_neutral(alpha,ind):
    vcrst = np.full_like(alpha,fill_value=np.nan)# maybe slow down
    ind_sum = dict()
    ind_num = dict()
    for kk in range(len(alpha)):
        v = alpha[kk]
        g = ind[kk]
        if not np.isinf(v) and not np.isnan(v) and not int_isnan(g) and g>=0: #TODO test nan ind and minus ind value
            if g not in ind_sum.keys():
                ind_sum[g] = v
                ind_num[g] = 1
            else:
                ind_sum[g] += v
                ind_num[g] += 1
    for g in ind_sum.keys():
        ind_sum[g] /= ind_num[g]
        
    for kk in range(len(alpha)):
        if not np.isinf(alpha[kk]) and not np.isnan(alpha[kk]) and not int_isnan(ind[kk]) and ind[kk]>=0: #TODO g is large minus number
            vcrst[kk] = alpha[kk] - ind_sum[ind[kk]]
        else:
            vcrst[kk] = np.nan
    return vcrst
            
            
def neutral(array, industry,market=False):
    aaret = np.full_like(array,fill_value=np.nan)
    if market == True:
        for di in range(array.shape[0]):
            aaret[di,:] = array[di, :] - array[np.where(~np.isnan(array[di, :])),:].mean()
    else:
        for di in range(array.shape[0]):
            aaret[di,:] = do_neutral(array[di, :], industry[di, :])
    return aaret

alpha = np.random.randn(2500,2500)
ind = np.random.randint(25, size=(2500, 2500))
neutral(alpha,ind,market=False)

CPU times: user 45.8 s, sys: 424 ms, total: 46.2 s
Wall time: 46.1 s


## pandas version

In [73]:
%%time
def industry_neutral(df_signal,df_ind):
    industry_ids = pd.unique(pd.Series(df_ind.values.ravel()).dropna())
    industry_ids.sort()
    l = []
    for industry_id  in industry_ids:
        df_industry_one = df_ind == industry_id
        df_industry_one = df_industry_one.replace(False,np.nan)
        df_signal_one_indstry = df_industry_one * df_signal
        sr_signal_sum_one_indstry = df_signal_one_indstry.mean(1)
        df_signal_one_indstry_neutral = df_signal_one_indstry.sub(sr_signal_sum_one_indstry.values,axis=0)
        l.append(df_signal_one_indstry_neutral.fillna(0))
    return sum(l)
df_alpha = pd.DataFrame(np.random.randn(2500,2500))
df_ind = pd.DataFrame(np.random.randint(25, size=(2500, 2500)) )
df1 = industry_neutral(df_alpha,df_ind)

CPU times: user 20.8 s, sys: 17.5 s, total: 38.3 s
Wall time: 22.4 s


## pandas-numpy version2

In [68]:
%%time
def industry_neutral(df_signal,df_ind):
    df_ind = df_ind.astype('float32').values
    df_signal = df_signal.values
    industry_ids = pd.unique(pd.Series(df_ind.ravel()).dropna())
    industry_ids.sort()
    l = []
    for industry_id  in industry_ids:
#         df_industry_one = df_ind == industry_id
#         df_industry_one = df_industry_one.astype('float32').replace(False,np.nan)
        np.place(df_ind,df_ind == industry_id,[np.nan])
        df_signal_one_indstry = df_ind * df_signal
        sr_signal_sum_one_indstry = np.nanmean(df_signal_one_indstry,1)
        df_signal_one_indstry_neutral = df_signal_one_indstry - sr_signal_sum_one_indstry
        l.append(np.nan_to_num(df_signal_one_indstry_neutral ))
    return sum(l)


df2 = industry_neutral(df_alpha,df_ind)

CPU times: user 10.8 s, sys: 8.77 s, total: 19.5 s
Wall time: 19.5 s


In [70]:
(df1.values == df2).all()

False

In [71]:
df1

,0,1,2,3,4,5,6,7,8,9,...,2490,2491,2492,2493,2494,2495,2496,2497,2498,2499
0,-0.138938,0.466766,-0.315825,1.880030,-1.173142,1.409573,0.012022,0.467400,0.554791,0.203615,...,1.329616,0.325483,-0.514283,0.529012,-0.033800,0.041032,-0.023428,0.405209,0.354724,-0.072836
1,0.233981,-1.169011,-0.393371,-1.093623,-0.434953,1.103897,-0.221288,0.897952,-0.728486,-0.146597,...,-0.288926,-0.306989,0.025081,1.691335,0.760523,-0.490425,1.204255,0.201295,-1.001775,-0.192172
2,-0.539499,0.445228,1.827668,-1.530336,1.531208,-1.254695,-0.478856,-1.384511,-1.068470,-0.655435,...,-1.119222,0.115412,-1.269341,-1.031644,0.573025,0.490734,1.006786,-0.270953,-0.202410,-0.578327
3,-1.364940,0.535926,1.782628,-1.562931,0.066462,0.503770,-0.660225,-0.447511,0.199888,-0.835722,...,0.682057,-0.642590,0.354693,0.496631,0.854038,0.010993,1.410007,-0.086335,0.804226,-0.735718
4,-1.141802,-0.241081,-0.759809,-1.046608,-0.562807,-0.604013,0.301110,-0.957961,1.149422,0.367759,...,-1.461222,-0.707998,-1.064673,-0.610473,-0.810304,-0.931266,0.807722,1.395834,-0.875060,1.187258
5,-1.184308,0.429968,-0.014340,0.382282,-0.463205,-1.139052,-0.025755,2.128211,-0.554537,0.630614,...,-1.589136,-0.276357,0.239042,1.421158,-1.243440,0.066039,0.534191,1.091273,1.093036,0.567799
6,1.582791,-0.301220,-0.721126,-0.447929,-1.198409,0.111634,1.698394,0.037109,2.351428,0.270329,...,0.360352,0.309287,1.092215,-0.127340,-0.351723,-1.418053,-2.110659,-0.525448,1.442649,-0.556680
7,2.004075,0.554375,-0.153361,-1.555681,-0.649636,0.058392,1.816617,-1.007288,1.148498,-1.855392,...,-0.680262,1.075627,-0.163520,-0.533486,-0.828389,0.594512,0.449698,1.044804,1.719224,0.072622
8,0.313119,0.650086,-0.311688,0.056528,0.523877,0.683543,1.366957,1.158152,0.212451,0.760516,...,1.692159,0.874050,1.043566,0.872375,0.484194,0.234117,-0.030858,0.176037,0.857209,1.424366
9,0.822539,-0.057481,0.632807,-0.270442,-0.158030,1.065352,0.079561,0.491619,-0.004531,0.351536,...,-0.859447,0.041101,-0.240439,0.118796,1.337896,-0.556810,0.063548,1.429395,1.450157,0.516413


In [72]:
df2

array([[  1.42582618e+02,  -8.25694641e-01,  -1.25799873e+02, ...,
          5.14848669e+01,   1.45628973e+02,  -1.16993664e+02],
       [ -1.08961429e+02,   5.77605188e+02,  -2.25654620e+00, ...,
          3.39126791e+01,  -1.54613607e+02,  -2.76855676e-01],
       [ -5.38085092e+02,   3.33183785e+01,   9.17606652e+00, ...,
         -1.83648917e+01,  -1.24243599e+02,  -1.62112671e+02],
       ..., 
       [  3.48403014e+02,   1.64090218e+01,   0.00000000e+00, ...,
          5.25159992e+02,   1.35720698e+03,  -2.00376544e+02],
       [  0.00000000e+00,   4.32565750e+02,  -3.40533686e+02, ...,
          1.05542662e+01,   1.64389442e+00,  -3.17759832e+02],
       [  7.10261809e+01,  -2.88617847e+00,   6.99359332e+02, ...,
         -9.33400423e+01,   6.91050767e+00,  -9.08673573e+01]])

In [60]:
df_alpha = pd.DataFrame(np.random.randn(2500,2500))
df_ind = pd.DataFrame(np.random.randint(25, size=(2500, 2500)) )
 

df_ind = df_ind.astype('float32').values
df_signal = df_alpha.values
industry_ids = pd.unique(pd.Series(df_ind.ravel()).dropna())
industry_ids.sort()

l = []
for industry_id  in industry_ids:
#         df_industry_one = df_ind == industry_id
#         df_industry_one = df_industry_one.astype('float32').replace(False,np.nan)
    np.place(df_ind,df_ind == industry_id,[np.nan])
    df_industry_one = df_ind
    df_signal_one_indstry = df_industry_one * df_signal
    sr_signal_sum_one_indstry = np.nanmean(df_signal_one_indstry,1)
    df_signal_one_indstry_neutral = df_signal_one_indstry - sr_signal_sum_one_indstry
    l.append(np.nan_to_num(df_signal_one_indstry_neutral ))
sum(l)

array([[ -1.52194445e+02,   0.00000000e+00,  -1.93285192e+00, ...,
         -8.72563052e+01,  -4.98352626e+01,   4.05530809e+01],
       [ -8.51405650e+01,  -5.43561921e+02,  -3.84196762e+02, ...,
          6.60477456e+01,  -2.68277032e+01,   7.78027548e+01],
       [ -2.34502493e+02,  -6.12947885e+02,  -1.21603825e+01, ...,
         -7.49301117e+02,  -6.55310608e-01,   7.71821737e+02],
       ..., 
       [  1.57770690e+01,  -1.83423198e+02,   2.66597026e+01, ...,
          7.32081209e+02,  -1.98639661e+01,   1.41731879e+03],
       [ -2.34206300e+02,   2.45571872e+02,   1.91831496e+02, ...,
         -1.69791136e+00,  -4.00176920e+02,   3.85673424e+01],
       [  8.26688872e+00,  -1.41184263e+01,  -5.06768331e+00, ...,
         -5.70372228e-01,   0.00000000e+00,  -2.55615093e+02]])

AttributeError: 'numpy.ndarray' object has no attribute 'nanmax'

In [39]:
df_ind


array([[24,  2, 24, ..., 16, 10, 11],
       [ 0, 23, 13, ..., 24,  9, 19],
       [ 4, 21, 14, ..., 18,  7, 10],
       ..., 
       [12, 23, 14, ...,  2, 15,  0],
       [20, 15, 17, ...,  6, 24,  6],
       [16,  0,  8, ...,  7, 21,  7]])

## use cython

In [9]:
%%time
%%cython -a
import numpy as np
cimport numpy as np
cimport cython
cdef extern from "math.h":
    int isnan(double x)
    

cdef int_isnan(int x):
    return isnan(float(x))
    
DTYPE = np.float32
ctypedef np.float32_t DTYPE_T 
ctypedef np.int32_t INDTYPE_T 



cdef do_neutral(np.ndarray[np.float32_t] alpha,np.ndarray[np.int32_t] ind):
    cdef int kk, jj
    cdef DTYPE_T v
    cdef INDTYPE_T g 
    cdef dict ind_sum = dict()
    cdef dict ind_num = dict()
    
    cdef np.ndarray[np.float32_t] vcrst = np.full_like(alpha,fill_value=np.nan,dtype=np.float32)# maybe slow down
    
#     ind_sum = dict()
#     ind_num = dict()
    for kk in xrange(len(alpha)):
        v = alpha[kk]
        g = ind[kk]
        if not np.isinf(v) and not np.isnan(v) and not int_isnan(g) and g>=0: #TODO test nan ind and minus ind value
            if g not in ind_sum.keys():
                ind_sum[g] = v
                ind_num[g] = 1
            else:
                ind_sum[g] += v
                ind_num[g] += 1
    for g in ind_sum.keys():
        ind_sum[g] /= ind_num[g]
        
    for kk in xrange(len(alpha)):
        if not np.isinf(alpha[kk]) and not np.isnan(alpha[kk]) and not int_isnan(ind[kk]) and ind[kk]>=0: #TODO g is large minus number
            vcrst[kk] = alpha[kk] - ind_sum[ind[kk]]
        else:
            vcrst[kk] = np.nan
    return vcrst
            
            
cdef neutral(np.ndarray[np.float32_t,ndim=2] array, np.ndarray[np.int32_t,ndim=2] industry,market=False):
    cdef np.ndarray[np.float32_t,ndim=2] aaret = np.full_like(array,fill_value=np.nan,dtype=np.float32)
    cdef int di
    if market == True:
        for di in xrange(array.shape[0]):
            aaret[di,:] = array[di, :] - array[np.where(~np.isnan(array[di, :])),:].mean()
    else:
        for di in xrange(array.shape[0]):
            aaret[di,:] = do_neutral(array[di, :], industry[di, :])
    return aaret

cdef main():
    cdef np.ndarray[np.float32_t,ndim=2] alpha = np.random.randn(2500,2500).astype(np.float32)
    cdef np.ndarray[np.int32_t,ndim=2] ind = np.random.randint(25, size=(2500, 2500),dtype=np.int32)
    neutral(alpha,ind,market=False)
    
main()

CPU times: user 23 s, sys: 232 ms, total: 23.3 s
Wall time: 25.1 s


## weave inline

In [ ]:
%%cython
import numpy as np
cimport numpy as np
cpdef foo(n):
    cdef np.ndarray[double] vola = np.empty(n, dtype=float)
    return vola

foo(4)

## pandas multiindex version

In [34]:
%%time
def industry_neutral(df_signal,df_ind):
    df_sig = pd.DataFrame(df_signal.unstack(),columns=['sig'])
    sr_ind = df_ind.unstack()
    df_sig['ind'] = sr_ind
    df_sig['sig_indmean']=df_sig.groupby(level=0).transform('mean')
    sr = df_sig['sig'] - df_sig['sig_indmean']
    return sr.unstack()
df_alpha = pd.DataFrame(np.random.randn(2500,2500))
df_ind = pd.DataFrame(np.random.randint(25, size=(2500, 2500)) )
df = industry_neutral(df_alpha,df_ind)

ValueError: Wrong number of items passed 2, placement implies 1